In [23]:
#initialization 
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

from warnings import simplefilter 
simplefilter(action='ignore', category=DeprecationWarning)

#importing qiskit
import qiskit
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info import Operator
from qiskit.circuit import Parameter, ParameterVector
from qiskit.optimization import QuadraticProgram

#import basic plot tools
from qiskit.visualization import plot_histogram, plot_state_city

In [17]:
def quadratic_program_from_graph(graph: nx.Graph) -> QuadraticProgram:
    '''Constructs a QuadraticProblem from a given graph for a MAXCUT instance.
    Args:
        graph: UNderlying graph of the problem
    Returns: 
        QuadraticProblem
    '''
    # get weight matrix from the graph
    weight_matrix = nx.adjacency_matrix(graph)
    shape = weight_matrix.shape
    size = shape[0]
    # build QUBO matrix Q from weight matrix
    qubo_matrix = np.zeros((size, size))
    qubo_vector = np.zeros(size)
    for i in range(size):
        for j in range(size):
            qubo_matrix[i,j] -= weight_matrix[i, j]
            qubo_vector[i] += weight_matrix[i,j]
    # define the quadraric program
    quadratic_program = QuadraticProgram('maxcut')
    for i in range(size):
        var_name = 'x_{}'.format(i)
        quadratic_program.binary_var(var_name)
    quadratic_program.minimize(quadratic=qubo_matrix, linear=qubo_vector, constant=0)
    #print(quadratic_program.export_as_lp_string())
    return quadratic_program

In [7]:
import networkx as nx

graph = nx.Graph()
# add nodes and edges
graph.add_nodes_from(np.arange(0,6,1))
edges = [(0,1,2.0), (0,2,3.0), (0,4,4.0), (0,5,1.0), (1,2,4.0), (1,3,1.0), (1,4,1.0), (1,5,3.0), (2,4,2.0), (2,5,3.0), (3,4,5.0), (3,5,1.0)]
graph.add_weighted_edges_from(edges)

In [31]:
qp = quadratic_program_from_graph(graph)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: maxcut

Minimize
 obj: 10 x_0 + 11 x_1 + 12 x_2 + 7 x_3 + 12 x_4 + 8 x_5 + [ - 8 x_0*x_1
      - 12 x_0*x_2 - 16 x_0*x_4 - 4 x_0*x_5 - 16 x_1*x_2 - 4 x_1*x_3 - 4 x_1*x_4
      - 12 x_1*x_5 - 8 x_2*x_4 - 12 x_2*x_5 - 20 x_3*x_4 - 4 x_3*x_5 ]/2
Subject To

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5
End



In [32]:
from qiskit.aqua.algorithms import QAOA
from qiskit.optimization.algorithms import admm_optimizer

#size = qp.get_num_vars()
#operator, _ = qp.to_ising()
#p = 1 # depth of the ansatz 


In [37]:
def qaoa_circuit(qubo: QuadraticProgram, p: int = 1):
    '''
    Given a QUBO instance and the number of layers, construct 
    a corresponding parametrized circuit with p layers.
    Args:
        qubo: The quadratic qubo instance
        p: The number of layers in the QAOA circuit
    Returns: 
        The parametrized QAOA circuit
    '''
    size = len(qubo.variables)
    qubo_matrix = qubo.objective.quadratic.to_array(symmetric=True)
    qubo_linearity = qubo.objective.linear.to_array()

    # Prepare the quantum and classical registers
    qaoa_circuit = QuantumCircuit(size, size)
    # Apply the initial layer of Hadamard gates to all qubits
    qaoa_circuit.h(range(size))

    # Create the parameters to be used in the circuit
    gammas = ParameterVector('gamma', p)
    betas = ParameterVector('beta', p)

    # Outer loop to create eahc layer
    for layer in range(p):
        # Apply Rz rotational gates from the cost layer
        for i in range(size):
            w_i = (qubo_linearity[layer] + sum(qubo_matrix[i])) / 2
            qaoa_circuit.rz(phi=gammas[layer]*w_i, qubit=i)
        # Apply the Rzz rotational gates for entangled qubits from cost layer
        for i in range(1,size):
            for j in range(i):
                w_ij = qubo_matrix[i,j]/4
                qaoa_circuit.rzz(theta=gammas[layer]*w_ij, qubit1=i, qubit2=j)
        # Apply single qubit Rx rotations from mixer layer to all qubits
        qaoa_circuit.barrier()
        for i in range(size):
            qaoa_circuit.rx(theta=2*betas[layer], qubit=i)

    # Measure the result in computational basis
    qaoa_circuit.barrier()
    qaoa_circuit.measure(range(size), range(size))

    return qaoa_circuit

qc = qaoa_circuit(qp)
qc.draw(fold=-1)

┌───┐      ┌───────┐                                                                                                                                                                                                                                                                                       ░ ┌───────────────┐ ░ ┌─┐               
q_0: ┤ H ├──────┤ RZ(0) ├───────■───────────────────■────────────────────────────────────────■────────────────────────────■────────────────────────────────────────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░─┤ RX(2*beta[0]) ├─░─┤M├───────────────
     ├───┤┌─────┴───────┴─────┐ │zz(-0.5*gamma[0])  │                                        │                            │                                                │                                                                                                                                    ░ ├───────────────┤ ░ └╥┘┌─┐            
q_1: ┤ H ├┤ RZ(-0.5*gamma[0]) ├─■───────────────────┼────────────────────■───────────────────┼───────■────────────────────┼───────────────────────────■────────────────────┼────────────────────────────────────────■───────────────────────────────────────────────────────────────────────────────────────────░─┤ RX(2*beta[0]) ├─░──╫─┤M├────────────
     ├───┤├───────────────────┤                     │zz(-0.75*gamma[0])  │zz(-1.0*gamma[0])  │       │                    │                           │                    │                                        │                                                                                           ░ ├───────────────┤ ░  ║ └╥┘┌─┐         
q_2: ┤ H ├┤ RZ(-1.0*gamma[0]) ├─────────────────────■────────────────────■───────────────────┼───────┼────────────────────┼───────────────────■───────┼────────────────────┼────────────────────■───────────────────┼─────────────────────────────────────────■─────────────────────────────────────────────────░─┤ RX(2*beta[0]) ├─░──╫──╫─┤M├─────────
     ├───┤└┬──────────────────┤                                                              │zz(0)  │zz(-0.25*gamma[0])  │                   │zz(0)  │                    │                    │                   │                                         │                                                 ░ ├───────────────┤ ░  ║  ║ └╥┘┌─┐      
q_3: ┤ H ├─┤ RZ(1.5*gamma[0]) ├──────────────────────────────────────────────────────────────■───────■────────────────────┼───────────────────■───────┼────────────────────┼────────────────────┼───────────────────┼────────────────────■────────────────────┼────────────────────■────────────────────────────░─┤ RX(2*beta[0]) ├─░──╫──╫──╫─┤M├──────
     ├───┤┌┴──────────────────┤                                                                                           │zz(-1.0*gamma[0])          │zz(-0.25*gamma[0])  │                    │zz(-0.5*gamma[0])  │                    │zz(-1.25*gamma[0])  │                    │                            ░ ├───────────────┤ ░  ║  ║  ║ └╥┘┌─┐   
q_4: ┤ H ├┤ RZ(-1.0*gamma[0]) ├───────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────■────────────────────┼────────────────────■───────────────────┼────────────────────■────────────────────┼────────────────────┼────────────────────■───────░─┤ RX(2*beta[0]) ├─░──╫──╫──╫──╫─┤M├───
     ├───┤└┬──────────────────┤                                                                                                                                            │zz(-0.25*gamma[0])                      │zz(-0.75*gamma[0])                       │zz(-0.75*gamma[0])  │zz(-0.25*gamma[0])  │zz(0)  ░ ├───────────────┤ ░  ║  ║  ║  ║ └╥┘┌─┐
q_5: ┤ H ├─┤ RZ(1.0*gamma[0]) ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────────

In [41]:
from qiskit.aqua.algorithms import QAOA
from qiskit.optimization.algorithms import ADMMOptimizer
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit.optimization.algorithms.admm_optimizer import ADMMOptimizationResult
backend = Aer.get_backend('statevector_simulator')
qaoa = QAOA(quantum_instance=backend, initial_point=[0.0, 0.0])
eigen_optimizer = MinimumEigenOptimizer(min_eigen_solver=qaoa)
qp = quadratic_program_from_graph(graph)
result = eigen_optimizer.solve(qp)

print(result)

optimal function value: 0.0
optimal value: [0. 0. 0. 0. 0. 0.]
status: SUCCESS
